In [ ]:
# %% [markdown]
# # Setup

# %%
# If needed (uncomment in Colab/clean env):
# !pip install -q "openai>=1.101.0" pandas tqdm nest_asyncio

# %%
import os
import json
import asyncio
import pandas as pd
from tqdm import tqdm

# Async OpenAI client (official)
from openai import AsyncOpenAI

# Jupyter event loop helper so we can call async code from normal cells
import nest_asyncio
nest_asyncio.apply()

# IMPORTANT: set your key as an environment variable rather than hardcoding:
# os.environ["OPENAI_API_KEY"] = "sk-..."   # <-- set externally (e.g., in your shell, .env, or here if you must)
aclient = AsyncOpenAI(api_key="...")


In [ ]:
# %% [markdown]
# # Load and prepare data (unchanged)

# %%
import pandas as pd
promises = pd.read_csv("../data/sxp1500_presentations_ceo_aggregated_promises_expanded_cleaned_transcriptlevel_horizon_specificity.csv")# create promise_id column, it is gvkey_transcriptid_2digitnumber (01, 02, 03, ...)
promises['promise_id'] = promises.groupby(['gvkey', 'transcriptid']).cumcount() + 1
promises['promise_id'] = promises['gvkey'].astype(str) + '_' + promises['transcriptid'].astype(str) + '_' + promises['promise_id'].apply(lambda x: f'{x:02d}')
promises_select = promises[['gvkey', 'mostimportantdateutc', 'companyname', 'exec_fullname', 'execid', 'promise_id','1-promise-verbatim' ,'2-promise-explain' ,'3-promise-horizon-v2', 'specificity_score']].sort_values(by=['gvkey', 'mostimportantdateutc',])

labels = pd.read_csv("promises_with_keywords_v5_labels.csv")
def revert_promise_id(promise_id):
    parts = promise_id.split('_')
    fixed_parts = []
    for part in parts:
        if part.endswith('.0'):
            part = str(int(float(part)))
        fixed_parts.append(part)
    return '_'.join(fixed_parts)

labels['promise_id'] = labels['promise_id'].apply(revert_promise_id)

labels = labels[['promise_id', 'primary_keyword']]

# merge promises and labels on promise_id
promises_select = pd.merge(labels, promises_select, on=['promise_id'], how='left')

# %%
promises_select.head(5)


/var/folders/qr/1pq07r555gxgt22gqksl2tlw0000gn/T/ipykernel_56829/2245440812.py:6: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  promises = pd.read_csv("../data/sxp1500_presentations_ceo_aggregated_promises_expanded_cleaned_transcriptlevel_horizon_specificity.csv")


,promise_id,primary_keyword,gvkey,mostimportantdateutc,companyname,exec_fullname,execid,1-promise-verbatim,2-promise-explain,3-promise-horizon-v2,specificity_score
0,10353_45212_05,Shareholder and Stakeholder Trust and Engagement,10353,2010-01-06,"Team, Inc.",Phillip J. Hawk,49854,We have always been and continue to be fully c...,The CEO asserts a continuous commitment to eth...,unclear,1.0
1,11600_45248_01,Risk Management,11600,2010-01-06,"Worthington Industries, Inc.",John P. McConnell,3497,But I assure you we will remain vigilant in se...,The CEO is committing that the company will ma...,unclear,1.0
2,25338_45259_01,Experience Enhancement,25338,2010-01-06,Bed Bath & Beyond Inc.,Steven H. Temares,13517,We always look for ways to enhance our custome...,The CEO is affirming a long‐term commitment to...,unclear,1.0
3,25338_45259_02,Investment,25338,2010-01-06,Bed Bath & Beyond Inc.,Steven H. Temares,13517,While we continue to review and prioritize our...,The CEO is committing to allocate necessary ca...,unclear,1.0
4,146017_45297_01,Pricing and Rate Management,146017,2010-01-06,"Acuity Brands, Inc.",Vernon J. Nagel,11057,"However, as I have said before, we will defend...",The CEO is committing the company to actively ...,unclear,1.0


In [3]:
# %% [markdown]
# # Use ChatGPT to Search (now async & concurrent; minimal code changes otherwise)

# %%
STATUS_MAP = {
    "ON_TIME": "delivered on time",
    "DELAYED": "delayed delivery",
    "NOT_DELIVERED": "not delivered",
    "UNMEASURABLE": "it is tough to define what a promise delivery is",
    "NO_EVIDENCE": "it is easy to define promise delivery but I couldn't find relevant data on it",
}

JSON_SCHEMA = {
    "name": "promise_audit",
    "strict": True,
    "schema": {
        "type": "object",
        "additionalProperties": False,
        "properties": {
            "status_code": {"type": "string", "enum": list(STATUS_MAP.keys())},
            "delivery": {"type": "string", "enum": list(STATUS_MAP.values())},
            "explanation": {"type": "string"},
            "sources": {
                "type": "array",
                "items": {"type": "string", "format": "uri"},
                "uniqueItems": True,
            },
        },
        "required": ["status_code", "delivery", "explanation", "sources"],
    },
}

INSTRUCTIONS = """You are a precise promise-auditor. Use web search before answering.
Classify delivery using these codes and labels (must match exactly):
ON_TIME → delivered on time
DELAYED → delayed delivery
NOT_DELIVERED → not delivered
UNMEASURABLE → it is tough to define what a promise delivery is
NO_EVIDENCE → it is easy to define promise delivery but I couldn't find relevant data on it

Rules:
- If promise has a concrete deliverable + deadline: ON_TIME / DELAYED / NOT_DELIVERED.
- If deliverable is concrete but you cannot find reliable evidence: NO_EVIDENCE.
- If the promise is vague/ongoing and not operationalizable: UNMEASURABLE.
Explain briefly (timeline logic + why). Provide sources.
Return one JSON object only, matching the schema.

JSON_SCHEMA = {
    "name": "promise_audit",
    "strict": True,
    "schema": {
        "type": "object",
        "additionalProperties": False,
        "properties": {
            "status_code": {"type": "string", "enum": list(STATUS_MAP.keys())},
            "delivery": {"type": "string", "enum": list(STATUS_MAP.values())},
            "explanation": {"type": "string"},
            "sources": {
                "type": "array",
                "items": {"type": "string", "format": "uri"},
                "uniqueItems": True,
            },
        },
        "required": ["status_code", "delivery", "explanation", "sources"],
    }

STATUS_MAP = {
    "ON_TIME": "delivered on time",
    "DELAYED": "delayed delivery",
    "NOT_DELIVERED": "not delivered",
    "UNMEASURABLE": "it is tough to define what a promise delivery is",
    "NO_EVIDENCE": "it is easy to define promise delivery but I couldn't find relevant data on it",
}
"""

def _row_payload(row: pd.Series) -> str:
    data = {
        "promise_id": row.get("promise_id"),
        "companyname": row.get("companyname"),
        "exec_fullname": row.get("exec_fullname"),
        "mostimportantdateutc": str(row.get("mostimportantdateutc")),
        "promise_verbatim": row.get("1-promise-verbatim"),
        "promise_explain": row.get("2-promise-explain"),
    }
    return (
        "Assess whether the following promise was delivered. Use web search. "
        "Return ONLY the JSON per schema.\n" + json.dumps(data, ensure_ascii=False)
    )


In [4]:
# %%
# --- ASYNC version of the per-row call (minimal change: just 'await' + AsyncOpenAI) ---

async def assess_promise_row_async(row: pd.Series, retries: int = 2, backoff: float = 1.5) -> dict:
    for i in range(retries + 1):
        try:
            r = await aclient.responses.create(
                model="gpt-5",
                instructions=INSTRUCTIONS,
                input=_row_payload(row),
                tools=[{"type": "web_search_preview"}],   # leave as-is (as in your original code)
                reasoning={"effort": "medium"},
                max_output_tokens=20000,
                # Tip: for stricter JSON you could add a response_format with JSON_SCHEMA, but keeping minimal changes
            )
            out = getattr(r, "output_json", None) or json.loads(r.output_text)
            return {
                "promise_id": row.get("promise_id"),
                "status_code": out["status_code"],
                "delivery": out["delivery"],
                "explanation": out["explanation"],
                "sources": out.get("sources", []),
            }
        except Exception as e:
            if i == retries:
                return {
                    "promise_id": row.get("promise_id"),
                    "status_code": "NO_EVIDENCE",
                    "delivery": STATUS_MAP["NO_EVIDENCE"],
                    "explanation": f"API/search error: {e}",
                    "sources": [],
                }
            await asyncio.sleep(backoff * (i + 1))


In [6]:
# %%
# --- Concurrent runner with bounded parallelism + a Jupyter-friendly wrapper ---

async def audit_promises_async(df: pd.DataFrame, concurrency: int = 8) -> pd.DataFrame:
    sem = asyncio.Semaphore(concurrency)
    rows_out = []

    async def _bounded(row):
        async with sem:
            # use a shallow copy so the async task never sees a mutated Series
            return await assess_promise_row_async(row.copy())

    tasks = [asyncio.create_task(_bounded(row)) for _, row in df.iterrows()]

    for fut in tqdm(asyncio.as_completed(tasks), total=len(tasks), desc="Auditing (async)"):
        result = await fut
        rows_out.append(result)

    return pd.DataFrame(rows_out)


def audit_promises(df: pd.DataFrame, concurrency: int = 8) -> pd.DataFrame:
    """
    Thin wrapper so you can call audit_promises(...) from a normal Jupyter cell
    without writing 'await ...'. Uses nest_asyncio to cooperate with the IPython loop.
    """
    loop = asyncio.get_event_loop()
    # With nest_asyncio applied, run_until_complete works fine in notebooks.
    return loop.run_until_complete(audit_promises_async(df, concurrency))


In [7]:
promises_select.head()

,promise_id,primary_keyword,gvkey,mostimportantdateutc,companyname,exec_fullname,execid,1-promise-verbatim,2-promise-explain,3-promise-horizon-v2,specificity_score
0,10353_45212_05,Shareholder and Stakeholder Trust and Engagement,10353,2010-01-06,"Team, Inc.",Phillip J. Hawk,49854,We have always been and continue to be fully c...,The CEO asserts a continuous commitment to eth...,unclear,1.0
1,11600_45248_01,Risk Management,11600,2010-01-06,"Worthington Industries, Inc.",John P. McConnell,3497,But I assure you we will remain vigilant in se...,The CEO is committing that the company will ma...,unclear,1.0
2,25338_45259_01,Experience Enhancement,25338,2010-01-06,Bed Bath & Beyond Inc.,Steven H. Temares,13517,We always look for ways to enhance our custome...,The CEO is affirming a long‐term commitment to...,unclear,1.0
3,25338_45259_02,Investment,25338,2010-01-06,Bed Bath & Beyond Inc.,Steven H. Temares,13517,While we continue to review and prioritize our...,The CEO is committing to allocate necessary ca...,unclear,1.0
4,146017_45297_01,Pricing and Rate Management,146017,2010-01-06,"Acuity Brands, Inc.",Vernon J. Nagel,11057,"However, as I have said before, we will defend...",The CEO is committing the company to actively ...,unclear,1.0


In [ ]:
# remove 10% of execids (random_state=42), then select 9% of the remaining execids
execids_all = promises_select['execid'].drop_duplicates()
execids_10pct = execids_all.sample(frac=0.1, random_state=42)
execids_remaining = execids_all[~execids_all.isin(execids_10pct)]
execids_sample = execids_remaining.sample(frac=0.09, random_state=24)
promises_select_10percent = promises_select[promises_select['execid'].isin(execids_sample)]

# %%
promises_select_10percent.head()


,promise_id,primary_keyword,gvkey,mostimportantdateutc,companyname,exec_fullname,execid,1-promise-verbatim,2-promise-explain,3-promise-horizon-v2,specificity_score
13,178507_45899_02,Investment,178507,2010-01-07,MSCI Inc.,Henry A. Fernandez,37930,We will also be making non-compensation relate...,The CEO commits to additional non-compensation...,6,3.0
14,30463_46109_01,Profitability,30463,2010-01-08,"The Greenbrier Companies, Inc.",William A. Furman,48434,"We believe in Europe for 2010, we will be prof...","In this statement, the CEO commits to achievin...",11,3.0
44,4723_46631_02,Unclustered,4723,2010-01-20,U.S. Bancorp,Richard K. Davis,12469,"Going forward, we will continue to actively as...",The CEO commits that the company will maintain...,unclear,1.0
52,2710_46690_04,Business Growth,2710,2010-01-07,"Constellation Brands, Inc.","Robert S. Sands, II",13570,Crown's 2010 business plan has already been ag...,The CEO commits that the Crown joint venture w...,unclear,1.0
98,24856_47032_03,Advanced Therapeutics Pipeline,24856,2010-01-26,"Gilead Sciences, Inc.","John C. Martin, Ph.D.",17643,I believe that this will be a very exciting ye...,The CEO is affirming that the company will ini...,5,4.0


In [10]:
promises_select_first_batch = promises_select[promises_select['execid'].isin(execids_10pct)]
# see if tehre is overlap in promise_id between promises_select_first_batch and promises_select_10percent
overlap_promise_ids = set(promises_select_first_batch['promise_id']) & set(promises_select_10percent['promise_id'])
print(f"Number of overlapping promise_ids: {len(overlap_promise_ids)}")
if overlap_promise_ids:
    print("Sample overlapping promise_ids:", list(overlap_promise_ids)[:5])
else:
    print("No overlap in promise_ids.")


Number of overlapping promise_ids: 0
No overlap in promise_ids.


In [11]:
# Run a small test sample (same API as before; now concurrent under the hood)
promises_select_10percent_results = audit_promises(promises_select_10percent, concurrency=25)



Auditing (async): 100%|██████████| 5733/5733 [4:47:40<00:00,  3.01s/it]  


In [6]:
# save promises_select_10percent_results to csv
promises_select_10percent_results.to_csv("promises_select_10percent_batch2_results.csv", index=False)

# %%
promises_select_10percent_results.head()



,promise_id,status_code,delivery,explanation,sources
0,6304_47870_01,UNMEASURABLE,it is tough to define what a promise delivery is,The 2010-01-28 earnings-call statement commits...,['https://www.roic.ai/quote/KLAC%3AUW/transcri...
1,14824_48607_02,ON_TIME,delivered on time,"Baseline: by Q4 2009, Cincinnati Financial had...",['https://media.corporate-ir.net/media_files/i...
2,141384_47695_03,UNMEASURABLE,it is tough to define what a promise delivery is,"The Jan 27, 2010 statement pledges a general c...",['https://www.globenewswire.com/news-release/2...
3,183324_47928_01,UNMEASURABLE,it is tough to define what a promise delivery is,The 2010-02-02 statement is an open‑ended comm...,['https://pmt.pennymac.com/news-events/press-r...
4,62921_48282_07,ON_TIME,delivered on time,"Promise (Feb 3, 2010) was to hear in 2010 abou...",['https://www.prnewswire.com/news-releases/neu...


In [7]:
promises_select_10percent_results = pd.read_csv("promises_select_10percent_batch2_results.csv")

# %%
promises_select_10percent_results.head()

# %%


,promise_id,status_code,delivery,explanation,sources
0,6304_47870_01,UNMEASURABLE,it is tough to define what a promise delivery is,The 2010-01-28 earnings-call statement commits...,['https://www.roic.ai/quote/KLAC%3AUW/transcri...
1,14824_48607_02,ON_TIME,delivered on time,"Baseline: by Q4 2009, Cincinnati Financial had...",['https://media.corporate-ir.net/media_files/i...
2,141384_47695_03,UNMEASURABLE,it is tough to define what a promise delivery is,"The Jan 27, 2010 statement pledges a general c...",['https://www.globenewswire.com/news-release/2...
3,183324_47928_01,UNMEASURABLE,it is tough to define what a promise delivery is,The 2010-02-02 statement is an open‑ended comm...,['https://pmt.pennymac.com/news-events/press-r...
4,62921_48282_07,ON_TIME,delivered on time,"Promise (Feb 3, 2010) was to hear in 2010 abou...",['https://www.prnewswire.com/news-releases/neu...


In [8]:
# value count for status_code
promises_select_10percent_results['status_code'].value_counts()

# %%


status_code
UNMEASURABLE     2465
ON_TIME          2187
NO_EVIDENCE       442
DELAYED           396
NOT_DELIVERED     243
Name: count, dtype: int64

In [9]:
# merge with promises_select_10percent on promise_id
promises_select_10percent_results_merged = pd.merge(promises_select_10percent_results, promises_select, on=['promise_id'], how='left')

# %%

promises_select_10percent_results_merged.head()


,promise_id,status_code,delivery,explanation,sources,primary_keyword,gvkey,mostimportantdateutc,companyname,exec_fullname,execid,1-promise-verbatim,2-promise-explain,3-promise-horizon-v2,specificity_score
0,6304_47870_01,UNMEASURABLE,it is tough to define what a promise delivery is,The 2010-01-28 earnings-call statement commits...,['https://www.roic.ai/quote/KLAC%3AUW/transcri...,Business Leverage Strategies,6304,2010-01-28,KLA Corporation,Richard P. Wallace,25626,Driving improved operational leverage remains ...,"In this statement, the CEO commits to converti...",unclear,1.0
1,14824_48607_02,ON_TIME,delivered on time,"Baseline: by Q4 2009, Cincinnati Financial had...",['https://media.corporate-ir.net/media_files/i...,Rollout Initiatives,14824,2010-02-04,Cincinnati Financial Corporation,Kenneth William Stecher,24439,"Last year, our staff successfully accelerated ...",The CEO is committing to maintaining the accel...,10,2.0
2,141384_47695_03,UNMEASURABLE,it is tough to define what a promise delivery is,"The Jan 27, 2010 statement pledges a general c...",['https://www.globenewswire.com/news-release/2...,Customer Relationship Management,141384,2010-01-27,"Align Technology, Inc.",Thomas M. Prescott,36420,We are completely committed helping those doct...,"In this promise, the CEO assures that the comp...",unclear,1.0
3,183324_47928_01,UNMEASURABLE,it is tough to define what a promise delivery is,The 2010-02-02 statement is an open‑ended comm...,['https://pmt.pennymac.com/news-events/press-r...,Acquisitions,183324,2010-02-02,PennyMac Mortgage Investment Trust,Stanford Lee Kurland,9210,We remain fully committed to evaluating and cl...,The CEO is asserting a firm commitment that PM...,unclear,1.0
4,62921_48282_07,ON_TIME,delivered on time,"Promise (Feb 3, 2010) was to hear in 2010 abou...",['https://www.prnewswire.com/news-releases/neu...,Clinical trial progress and milestones,62921,2010-02-03,"Neurocrine Biosciences, Inc.","Kevin C. Gorman, Ph.D.",60255,we're going to hear this year on our lead CRF ...,The CEO promises that there will be an update ...,10,2.0


In [10]:
# value count for status_code based on different specificity_score
promises_select_10percent_results_merged.groupby(['status_code', 'specificity_score']).size().unstack()

# %%




specificity_score,1.0,2.0,3.0,4.0,5.0
status_code,,,,,
DELAYED,9,35,110,136,106
NOT_DELIVERED,25,52,47,56,63
NO_EVIDENCE,25,66,132,106,113
ON_TIME,184,448,615,511,429
UNMEASURABLE,1879,444,96,34,12


In [11]:
# value counts for status code for specificity score above 3
promises_select_10percent_results_merged[promises_select_10percent_results_merged['specificity_score'] > 2]['status_code'].value_counts()

status_code
ON_TIME          1555
DELAYED           352
NO_EVIDENCE       351
NOT_DELIVERED     166
UNMEASURABLE      142
Name: count, dtype: int64

In [12]:
# just the above but in percentages
promises_select_10percent_results_merged[promises_select_10percent_results_merged['specificity_score'] > 2]['status_code'].value_counts() / len(promises_select_10percent_results_merged[promises_select_10percent_results_merged['specificity_score'] > 2])

status_code
ON_TIME          0.606002
DELAYED          0.137178
NO_EVIDENCE      0.136789
NOT_DELIVERED    0.064692
UNMEASURABLE     0.055339
Name: count, dtype: float64

In [13]:
# show the table above, but percentages for each specificity_score
counts = promises_select_10percent_results_merged.groupby(['status_code', 'specificity_score']).size().unstack(fill_value=0)
col_totals = counts.sum(axis=0)
percentages = counts.div(col_totals, axis=1) * 100

# show total percent size of promises with each specificity score
total_promises = counts.values.sum()
specificity_percents = (col_totals / total_promises) * 100
print("Percent of promises with each specificity score:")
print(specificity_percents)

percentages


Percent of promises with each specificity score:
specificity_score
1.0    37.013780
2.0    18.227804
3.0    17.442875
4.0    14.704343
5.0    12.611198
dtype: float64


specificity_score,1.0,2.0,3.0,4.0,5.0
status_code,,,,,
DELAYED,0.424128,3.349282,11.0,16.132859,14.661134
NOT_DELIVERED,1.178134,4.976077,4.7,6.642942,8.713693
NO_EVIDENCE,1.178134,6.315789,13.2,12.574140,15.629322
ON_TIME,8.671065,42.870813,61.5,60.616845,59.336100
UNMEASURABLE,88.548539,42.488038,9.6,4.033215,1.659751


# Merge with previous batch

In [5]:
promises_select_10percent_results = pd.read_csv("promises_select_10percent_batch2_results.csv")
promises_select_10percent_results_merged = pd.merge(promises_select_10percent_results, promises_select, on=['promise_id'], how='left')


In [3]:
promises_select_10percent_results_batch1 = pd.read_csv("promises_select_10percent_results.csv")
# merge with promises_select_10percent on promise_id
promises_select_10percent_results_merged_batch1 = pd.merge(promises_select_10percent_results_batch1, promises_select, on=['promise_id'], how='left')

# %%

promises_select_10percent_results_merged_batch1.head()




,promise_id,status_code,delivery,explanation,sources,primary_keyword,gvkey,mostimportantdateutc,companyname,exec_fullname,execid,1-promise-verbatim,2-promise-explain,3-promise-horizon-v2,specificity_score
0,9203_47104_01,UNMEASURABLE,it is tough to define what a promise delivery is,"Promise made on January 27, 2010 is a broad, o...",['https://www.sec.gov/Archives/edgar/data/1024...,Efficiency,9203,2010-01-27,"Rockwell Automation, Inc.",Keith D. Nosbusch,17969,We will continue to effectively manage our cos...,"In this statement, the CEO commits to maintain...",unclear,1.0
1,25434_46799_01,UNMEASURABLE,it is tough to define what a promise delivery is,"The 2010 pledge is an ongoing ethos (""earn our...",['https://special.seattletimes.com/o/html/coff...,Customer Relationship Management,25434,2010-01-20,Starbucks Corporation,Howard D. Schultz,887,"Simply stated, we know that we have to earn ou...",The CEO is pledging that Starbucks will contin...,unclear,1.0
2,145046_47865_02,UNMEASURABLE,it is tough to define what a promise delivery is,The statement is an open‑ended operating philo...,['https://www.thestreet.com/investing/stocks/w...,Pricing and Rate Management,145046,2010-01-27,"Elevance Health, Inc.",Angela Rose Fick Braly,32716,We price our business in an actuarially sound ...,This statement commits the company to maintain...,unclear,1.0
3,3863_47450_01,UNMEASURABLE,it is tough to define what a promise delivery is,"On January 28, 2010, the CEO pledged that in 2...",['https://investors.deluxe.com/news-events/pre...,Cost Reduction Initiatives,3863,2010-01-28,Deluxe Corporation,Lee J. Schram,31329,"In 2010, we will not take our eyes off of cost...",The CEO commits that throughout 2010 the compa...,11,1.0
4,9850_47202_01,ON_TIME,delivered on time,"Promise (Jan 27, 2010): complete a 100‑MW biom...",['https://www.power-technology.com/marketdata/...,Clean and Renewable Energy Transition,9850,2010-01-27,The Southern Company,David M. Ratcliffe,19239,We are moving forward with construction of the...,The CEO is committing Southern Power to build ...,35,5.0


In [6]:
# concat the two dataframes
promises_select_10percent_results_merged_all = pd.concat([promises_select_10percent_results_merged_batch1, promises_select_10percent_results_merged])

# %%

promises_select_10percent_results_merged_all.head()


,promise_id,status_code,delivery,explanation,sources,primary_keyword,gvkey,mostimportantdateutc,companyname,exec_fullname,execid,1-promise-verbatim,2-promise-explain,3-promise-horizon-v2,specificity_score
0,9203_47104_01,UNMEASURABLE,it is tough to define what a promise delivery is,"Promise made on January 27, 2010 is a broad, o...",['https://www.sec.gov/Archives/edgar/data/1024...,Efficiency,9203,2010-01-27,"Rockwell Automation, Inc.",Keith D. Nosbusch,17969,We will continue to effectively manage our cos...,"In this statement, the CEO commits to maintain...",unclear,1.0
1,25434_46799_01,UNMEASURABLE,it is tough to define what a promise delivery is,"The 2010 pledge is an ongoing ethos (""earn our...",['https://special.seattletimes.com/o/html/coff...,Customer Relationship Management,25434,2010-01-20,Starbucks Corporation,Howard D. Schultz,887,"Simply stated, we know that we have to earn ou...",The CEO is pledging that Starbucks will contin...,unclear,1.0
2,145046_47865_02,UNMEASURABLE,it is tough to define what a promise delivery is,The statement is an open‑ended operating philo...,['https://www.thestreet.com/investing/stocks/w...,Pricing and Rate Management,145046,2010-01-27,"Elevance Health, Inc.",Angela Rose Fick Braly,32716,We price our business in an actuarially sound ...,This statement commits the company to maintain...,unclear,1.0
3,3863_47450_01,UNMEASURABLE,it is tough to define what a promise delivery is,"On January 28, 2010, the CEO pledged that in 2...",['https://investors.deluxe.com/news-events/pre...,Cost Reduction Initiatives,3863,2010-01-28,Deluxe Corporation,Lee J. Schram,31329,"In 2010, we will not take our eyes off of cost...",The CEO commits that throughout 2010 the compa...,11,1.0
4,9850_47202_01,ON_TIME,delivered on time,"Promise (Jan 27, 2010): complete a 100‑MW biom...",['https://www.power-technology.com/marketdata/...,Clean and Renewable Energy Transition,9850,2010-01-27,The Southern Company,David M. Ratcliffe,19239,We are moving forward with construction of the...,The CEO is committing Southern Power to build ...,35,5.0


In [7]:
# show the table above, but percentages for each specificity_score
counts = promises_select_10percent_results_merged_all.groupby(['status_code', 'specificity_score']).size().unstack(fill_value=0)
col_totals = counts.sum(axis=0)
percentages = counts.div(col_totals, axis=1) * 100

# show total percent size of promises with each specificity score
total_promises = counts.values.sum()
specificity_percents = (col_totals / total_promises) * 100
print("Percent of promises with each specificity score:")
print(specificity_percents)

percentages


Percent of promises with each specificity score:
specificity_score
1.0    33.721376
2.0    17.752929
3.0    18.917056
4.0    16.236501
5.0    13.372138
dtype: float64


specificity_score,1.0,2.0,3.0,4.0,5.0
status_code,,,,,
DELAYED,0.386100,3.106126,11.821862,16.084906,13.516609
NOT_DELIVERED,1.635249,4.659189,4.939271,6.084906,9.507446
NO_EVIDENCE,0.931183,5.565142,11.336032,12.028302,13.974800
ON_TIME,10.447422,46.246764,62.064777,62.735849,61.053837
UNMEASURABLE,86.600045,40.422778,9.838057,3.066038,1.947308


In [8]:
# only look at promises where primary_keyword is "Launch Announcements"
launch_announcements = promises_select_10percent_results_merged_all[
    promises_select_10percent_results_merged_all['primary_keyword'] == "Launch Announcements"
]

counts = launch_announcements.groupby(['status_code', 'specificity_score']).size().unstack(fill_value=0)
col_totals = counts.sum(axis=0)
percentages = counts.div(col_totals, axis=1) * 100

total_promises = counts.values.sum()
specificity_percents = (col_totals / total_promises) * 100
print("Percent of promises with each specificity score (Launch Announcements only):")
print(specificity_percents)

percentages

Percent of promises with each specificity score (Launch Announcements only):
specificity_score
1.0     2.808989
2.0    10.814607
3.0    36.235955
4.0    37.921348
5.0    12.219101
dtype: float64


specificity_score,1.0,2.0,3.0,4.0,5.0
status_code,,,,,
DELAYED,0.0,3.896104,12.403101,18.518519,11.494253
NOT_DELIVERED,5.0,3.896104,1.550388,3.333333,2.298851
NO_EVIDENCE,5.0,3.896104,11.240310,10.370370,16.091954
ON_TIME,25.0,59.740260,71.317829,67.407407,70.114943
UNMEASURABLE,65.0,28.571429,3.488372,0.370370,0.000000


In [17]:
# percentage value counts for status code for specificity score above 2
promises_select_10percent_results_merged[promises_select_10percent_results_merged['specificity_score'] > 2]['status_code'].value_counts(normalize=True) * 100

status_code
ON_TIME          60.600156
DELAYED          13.717849
NO_EVIDENCE      13.678878
NOT_DELIVERED     6.469213
UNMEASURABLE      5.533905
Name: proportion, dtype: float64

In [9]:
promises_select_10percent_results_merged[
    (promises_select_10percent_results_merged['specificity_score'] > 2) &
    (promises_select_10percent_results_merged['primary_keyword'] == "Launch Announcements")
]['status_code'].value_counts(normalize=True) * 100

status_code
ON_TIME          64.957265
DELAYED          16.666667
NO_EVIDENCE      15.384615
NOT_DELIVERED     1.709402
UNMEASURABLE      1.282051
Name: proportion, dtype: float64

In [10]:
all_avg_spec = promises_select_10percent_results_merged_all['specificity_score'].mean()
launch_avg_spec = promises_select_10percent_results_merged_all[
    promises_select_10percent_results_merged_all['primary_keyword'] == "Launch Announcements"
]['specificity_score'].mean()

print(f"Average specificity_score (all promises): {all_avg_spec:.2f}")
print(f"Average specificity_score (Launch Announcements): {launch_avg_spec:.2f}")


Average specificity_score (all promises): 2.58
Average specificity_score (Launch Announcements): 3.46
